In [2]:
import psycopg as pg
import time

# Connect to an existing QuestDB instance

conn_str = 'user=admin password=quest host=host.docker.internal port=8812 dbname=qdb'
with pg.connect(conn_str, autocommit=True) as connection:

    # Open a cursor to perform database operations

    with connection.cursor() as cur:

        #Query the database and obtain data as Python objects.

        cur.execute('SELECT * FROM github_events;')
        records = cur.fetchall()
        for row in records:
            print(row)

# the connection is now closed


('iNeil77/transformers', 'PullRequestEvent', 'iNeil77', datetime.datetime(2024, 2, 2, 1, 6, 46, 229749))
('fsadg32/repo14', 'PushEvent', 'flo2py', datetime.datetime(2024, 2, 2, 1, 6, 46, 231362))
('onnx/onnx', 'PushEvent', 'github-merge-queue[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231385))
('felix-kaestner/eslint-config', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231614))
('aws/aws-iot-device-sdk-cpp-v2', 'CreateEvent', 'alfred2g', datetime.datetime(2024, 2, 2, 1, 6, 46, 231636))
('bbcvc/jk', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231703))
('ilandalmut/ilandalmut', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231718))
('woshishiq1/gao', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231747))
('happyendermangit/discord-datamining', 'PushEvent', 'github-actions[bot]', datetime.datetime(2024, 2, 2, 1, 6, 46, 231765))
('BR-lovecraft/upptime', 'PushEvent